<a href="https://colab.research.google.com/github/loridlo-pixel/podtekst-site/blob/main/Retrieval_based_Voice_Conversion_WebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Retrieval-based-Voice-Conversion-WebUI](https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI) Training notebook

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RVC-Project/Retrieval-based-Voice-Conversion-WebUI/blob/main/Retrieval_based_Voice_Conversion_WebUI.ipynb)

In [ ]:
# @title 查看显卡
!nvidia-smi

In [ ]:
# @title 挂载谷歌云盘

from google.colab import drive

drive.mount("/content/drive")

In [ ]:
# @title 安装依赖
!apt-get -y install build-essential python3-dev ffmpeg
!pip3 install --upgrade setuptools wheel
!pip3 install --upgrade pip
!pip3 install faiss-cpu==1.7.2 fairseq gradio==3.14.0 ffmpeg ffmpeg-python praat-parselmouth pyworld numpy==1.23.5 numba==0.56.4 librosa==0.9.2

In [ ]:
# @title 克隆仓库

!git clone --depth=1 -b stable https://github.com/fumiama/Retrieval-based-Voice-Conversion-WebUI
%cd /content/Retrieval-based-Voice-Conversion-WebUI
!mkdir -p pretrained uvr5_weights

In [ ]:
# @title 更新仓库（一般无需执行）
!git pull

In [ ]:
# @title 安装aria2
!apt -y install -qq aria2

In [ ]:
# @title 下载底模
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o D48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o G48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0D48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0G48k.pth

In [ ]:
# @title 下载人声分离模型
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP2-人声vocals+非人声instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP2-人声vocals+非人声instrumentals.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP5-主旋律人声vocals+其他instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP5-主旋律人声vocals+其他instrumentals.pth

In [ ]:
# @title 下载hubert_base
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt -d /content/Retrieval-based-Voice-Conversion-WebUI -o hubert_base.pt

In [ ]:
# @title #下载rmvpe模型
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/rmvpe.pt -d /content/Retrieval-based-Voice-Conversion-WebUI -o rmvpe.pt

In [ ]:
# @title 从谷歌云盘加载打包好的数据集到/content/dataset

# @markdown 数据集位置
DATASET = (
    "/content/drive/MyDrive/dataset/lulu20230327_32k.zip"  # @param {type:"string"}
)

!mkdir -p /content/dataset
!unzip -d /content/dataset -B {DATASET}

In [ ]:
# @title 重命名数据集中的重名文件
!ls -a /content/dataset/
!rename 's/(\w+)\.(\w+)~(\d*)/$1_$3.$2/' /content/dataset/*.*~*

In [ ]:
# @title 启动web
%cd /content/Retrieval-based-Voice-Conversion-WebUI
# %load_ext tensorboard
# %tensorboard --logdir /content/Retrieval-based-Voice-Conversion-WebUI/logs
!python3 infer-web.py --colab --pycmd python3

In [ ]:
# @title 手动将训练后的模型文件备份到谷歌云盘
# @markdown 需要自己查看logs文件夹下模型的文件名，手动修改下方命令末尾的文件名

# @markdown 模型名
MODELNAME = "lulu"  # @param {type:"string"}
# @markdown 模型epoch
MODELEPOCH = 9600  # @param {type:"integer"}

!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth /content/drive/MyDrive/{MODELNAME}_D_{MODELEPOCH}.pth
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth /content/drive/MyDrive/{MODELNAME}_G_{MODELEPOCH}.pth
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/added_*.index /content/drive/MyDrive/
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/total_*.npy /content/drive/MyDrive/

!cp /content/Retrieval-based-Voice-Conversion-WebUI/weights/{MODELNAME}.pth /content/drive/MyDrive/{MODELNAME}{MODELEPOCH}.pth

In [ ]:
# @title 从谷歌云盘恢复pth
# @markdown 需要自己查看logs文件夹下模型的文件名，手动修改下方命令末尾的文件名

# @markdown 模型名
MODELNAME = "lulu"  # @param {type:"string"}
# @markdown 模型epoch
MODELEPOCH = 7500  # @param {type:"integer"}

!mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}

!cp /content/drive/MyDrive/{MODELNAME}_D_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth
!cp /content/drive/MyDrive/{MODELNAME}_G_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth
!cp /content/drive/MyDrive/*.index /content/
!cp /content/drive/MyDrive/*.npy /content/
!cp /content/drive/MyDrive/{MODELNAME}{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/weights/{MODELNAME}.pth

In [ ]:
# @title 手动预处理（不推荐）
# @markdown 模型名
MODELNAME = "lulu"  # @param {type:"string"}
# @markdown 采样率
BITRATE = 48000  # @param {type:"integer"}
# @markdown 使用的进程数
THREADCOUNT = 8  # @param {type:"integer"}

!python3 trainset_preprocess_pipeline_print.py /content/dataset {BITRATE} {THREADCOUNT} logs/{MODELNAME} True

In [ ]:
# @title 手动提取特征（不推荐）
# @markdown 模型名
MODELNAME = "lulu"  # @param {type:"string"}
# @markdown 使用的进程数
THREADCOUNT = 8  # @param {type:"integer"}
# @markdown 音高提取算法
ALGO = "harvest"  # @param {type:"string"}

!python3 extract_f0_print.py logs/{MODELNAME} {THREADCOUNT} {ALGO}

!python3 extract_feature_print.py cpu 1 0 0 logs/{MODELNAME} True

In [ ]:
# @title 手动训练（不推荐）
# @markdown 模型名
MODELNAME = "lulu"  # @param {type:"string"}
# @markdown 使用的GPU
USEGPU = "0"  # @param {type:"string"}
# @markdown 批大小
BATCHSIZE = 32  # @param {type:"integer"}
# @markdown 停止的epoch
MODELEPOCH = 3200  # @param {type:"integer"}
# @markdown 保存epoch间隔
EPOCHSAVE = 100  # @param {type:"integer"}
# @markdown 采样率
MODELSAMPLE = "48k"  # @param {type:"string"}
# @markdown 是否缓存训练集
CACHEDATA = 1  # @param {type:"integer"}
# @markdown 是否仅保存最新的ckpt文件
ONLYLATEST = 0  # @param {type:"integer"}

!python3 train_nsf_sim_cache_sid_load_pretrain.py -e lulu -sr {MODELSAMPLE} -f0 1 -bs {BATCHSIZE} -g {USEGPU} -te {MODELEPOCH} -se {EPOCHSAVE} -pg pretrained/f0G{MODELSAMPLE}.pth -pd pretrained/f0D{MODELSAMPLE}.pth -l {ONLYLATEST} -c {CACHEDATA}

In [ ]:
# @title 删除其它pth，只留选中的（慎点，仔细看代码）
# @markdown 模型名
MODELNAME = "lulu"  # @param {type:"string"}
# @markdown 选中模型epoch
MODELEPOCH = 9600  # @param {type:"integer"}

!echo "备份选中的模型。。。"
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth /content/{MODELNAME}_D_{MODELEPOCH}.pth
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth /content/{MODELNAME}_G_{MODELEPOCH}.pth

!echo "正在删除。。。"
!ls /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}
!rm /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/*.pth

!echo "恢复选中的模型。。。"
!mv /content/{MODELNAME}_D_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth
!mv /content/{MODELNAME}_G_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth

!echo "删除完成"
!ls /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}

In [ ]:
# @title 清除项目下所有文件，只留选中的模型（慎点，仔细看代码）
# @markdown 模型名
MODELNAME = "lulu"  # @param {type:"string"}
# @markdown 选中模型epoch
MODELEPOCH = 9600  # @param {type:"integer"}

!echo "备份选中的模型。。。"
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth /content/{MODELNAME}_D_{MODELEPOCH}.pth
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth /content/{MODELNAME}_G_{MODELEPOCH}.pth

!echo "正在删除。。。"
!ls /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}
!rm -rf /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/*

!echo "恢复选中的模型。。。"
!mv /content/{MODELNAME}_D_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth
!mv /content/{MODELNAME}_G_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth

!echo "删除完成"
!ls /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}

In [ ]:
# ===========================
# 1️⃣ Скачиваем RVC WebUI
# ===========================
!git clone https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI.git
import os
os.chdir("/content/Retrieval-based-Voice-Conversion-WebUI")

# ===========================
# 2️⃣ Устанавливаем зависимости
# ===========================
!pip install -q -r requirements.txt
!pip install -q torch torchaudio librosa scipy onnx onnxruntime

# ===========================
# 3️⃣ Подключаем GPU
# ===========================
import torch
print("GPU доступен:", torch.cuda.is_available())

# ===========================
# 4️⃣ Загружаем HuBERT и датасет
# ===========================
# Загрузите свои файлы HuBERT .onnx в /content/hubert-onnx/hubert_base.onnx
# Загрузите свои WAV-файлы в /content/mydataset
dataset_path = "/content/mydataset"
hubert_path = "/content/hubert-onnx/hubert_base.onnx"
os.makedirs("/content/hubert-onnx", exist_ok=True)
os.makedirs("/content/mydataset", exist_ok=True)

# ===========================
# 5️⃣ Создаём filelist.txt
# ===========================
filelist_path = "/content/filelist.txt"
wav_files = [f for f in os.listdir(dataset_path) if f.endswith(".wav")]
with open(filelist_path, "w") as f:
    for file in wav_files:
        f.write(f"{dataset_path}/{file}|0\n")
print(f"✅ filelist.txt создан, строк: {len(wav_files)}")

# ===========================
# 6️⃣ Создаём папку для чекпоинтов
# ===========================
output_dir = "/content/checkpoints_rvc"
os.makedirs(output_dir, exist_ok=True)

# ===========================
# 7️⃣ Запуск обучения RVC
# ===========================
!python infer/modules/train/train.py \
-se 5 -te 50 -e {output_dir} \
-sr 44100 -v v1 -f0 True -l False -c False \
--train_files {filelist_path} \
--hubert_model_path {hubert_path}


In [ ]:
import torch
torch.cuda.is_available()
!nvidia-smi


In [ ]:
!python infer/modules/train/preprocess.py
!python infer/modules/train/extract_feature_print.py


In [ ]:
!pwd
!ls


In [ ]:
%cd /content
!git clone https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI.git


In [ ]:
!ls


In [ ]:
!ls Retrieval-based-Voice-Conversion-WebUI/infer/modules/train


In [ ]:
%cd /content/Retrieval-based-Voice-Conversion-WebUI


In [ ]:
!python infer/modules/train/preprocess.py \
  /content/ \
  40000 \
  2 \
  my_experiment \
  False



In [ ]:
!ls /content/wavs


In [ ]:
# создаём папку для RVC
!mkdir /content/wavs

# перемещаем все WAV из корня в папку
!mv /content/*.wav /content/wavs/


In [ ]:
from google.colab import files
uploaded = files.upload()  # выбрать WAV

import os
os.makedirs("/content/wavs", exist_ok=True)

for name in uploaded.keys():
    os.rename(name, f"/content/wavs/{name}")


In [ ]:
!python infer/modules/train/preprocess.py /content/wavs 40000 2 my_experiment False


In [ ]:
# =========================
# 0. БАЗОВАЯ ПОДГОТОВКА
# =========================
import os

%cd /content
!apt-get update -y
!apt-get install -y ffmpeg

!pip install --upgrade pip setuptools wheel --no-cache-dir

# =========================
# 1. TORCH (CUDA 11.8 — оптимально для Colab)
# =========================
!pip install torch torchvision torchaudio \
  --index-url https://download.pytorch.org/whl/cu118 \
  --no-cache-dir

# =========================
# 2. КРИТИЧНЫЕ ЗАВИСИМОСТИ RVC
# =========================
!pip install \
  ffmpeg-python \
  av \
  librosa \
  soundfile \
  numpy \
  scipy \
  tqdm \
  numba \
  faiss-cpu \
  praat-parselmouth \
  pyworld \
  noisereduce \
  tensorboard \
  --no-cache-dir

# =========================
# 3. КЛОНИРУЕМ RVC
# =========================
RVC_PATH = "/content/Retrieval-based-Voice-Conversion-WebUI"

if not os.path.exists(RVC_PATH):
    !git clone https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI.git
else:
    print("ℹ️ RVC уже клонирован")

# =========================
# 4. СОЗДАЁМ ПАПКИ
# =========================
os.makedirs("/content/wavs", exist_ok=True)
os.makedirs(f"{RVC_PATH}/logs", exist_ok=True)

# =========================
# 5. ЗАГРУЗКА WAV
# =========================
from google.colab import files

uploaded = files.upload()

for name in uploaded.keys():
    dst = f"/content/wavs/{name}"
    if os.path.exists(dst):
        os.remove(dst)
    os.rename(name, dst)

print("✅ WAV файлы загружены в /content/wavs")

# =========================
# 6. ПРОВЕРКА TRAIN-МОДУЛЕЙ
# =========================
train_base = f"{RVC_PATH}/infer/modules/train"

needed = [
    "preprocess.py",
    "extract_feature_print.py",
    "train.py"
]

missing = [f for f in needed if not os.path.exists(f"{train_base}/{f}")]

if missing:
    print("❌ Отсутствуют файлы:", missing)
else:
    print("✅ Все train-файлы на месте")

# =========================
# 7. ПРОВЕРКА КРИТИЧНЫХ ИМПОРТОВ
# =========================
import av, ffmpeg, numpy, torch

print("✅ av / ffmpeg / numpy / torch импортируются")
print("🔥 СИСТЕМА ГОТОВА К PREPROCESS / TRAIN")
print("CUDA:", torch.cuda.is_available())


In [ ]:
import os
import librosa
import soundfile as sf
from pathlib import Path

SRC_DIR = "/content/wavs"
DST_DIR = "/content/Retrieval-based-Voice-Conversion-WebUI/dataset_speaker"
os.makedirs(DST_DIR, exist_ok=True)

filelist_path = os.path.join(DST_DIR, "filelist.txt")
filelist = []

TARGET_SR = 44100
SPEAKER = "spk1"

print("ℹ️ Начинаем безопасный preprocess WAV...")

for i, fname in enumerate(sorted(os.listdir(SRC_DIR))):
    if not fname.lower().endswith(".wav"):
        continue

    src_path = os.path.join(SRC_DIR, fname)
    y, sr = librosa.load(src_path, sr=TARGET_SR, mono=True)
    y = y / max(abs(y)) * 0.99  # нормализация

    dst_fname = f"{SPEAKER}_{i:04d}.wav"
    dst_path = os.path.join(DST_DIR, dst_fname)
    sf.write(dst_path, y, TARGET_SR)

    filelist.append(f"{dst_path}|{SPEAKER}")

# Создаём filelist.txt
with open(filelist_path, "w", encoding="utf-8") as f:
    f.write("\n".join(filelist))

print(f"✅ Обработано {len(filelist)} файлов")
print(f"✅ filelist.txt создан в {filelist_path}")
print("🔥 Preprocess завершён. Данные готовы к feature extraction / training")


In [ ]:
import os
import torch
import torchaudio
from pathlib import Path

# Пути
DATASET_DIR = "/content/Retrieval-based-Voice-Conversion-WebUI/dataset_speaker"
FEATURE_DIR = os.path.join(DATASET_DIR, "features")
os.makedirs(FEATURE_DIR, exist_ok=True)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"ℹ️ Feature extraction на {DEVICE}")

# Скачиваем HuBERT
HUBERT_PATH = "/content/hubert_base.pt"
if not os.path.exists(HUBERT_PATH):
    print("ℹ️ Скачиваем HuBERT base...")
    !wget -O {HUBERT_PATH} "https://huggingface.co/s3prl/hubert-base-ls960/resolve/main/hubert_base.pt"
    print("✅ HuBERT скачан")

# Загружаем модель через s3prl hubert
hubert_model = torch.hub.load("s3prl/s3prl", "hubert", source="github")
hubert_model.eval().to(DEVICE)

# Берём все WAV
wav_files = sorted([f for f in os.listdir(DATASET_DIR) if f.endswith(".wav")])
if not wav_files:
    raise FileNotFoundError("❌ WAV файлы не найдены в dataset_speaker!")

def extract_feature(wav_path):
    waveform, sr = torchaudio.load(wav_path)
    if sr != 16000:
        waveform = torchaudio.functional.resample(waveform, sr, 16000)
    waveform = waveform.mean(dim=0, keepdim=True)  # моно
    with torch.no_grad():
        feats = hubert_model(waveform.to(DEVICE))
    return feats.squeeze(0).cpu()

# Генерируем .pt файлы
for wav_name in wav_files:
    wav_path = os.path.join(DATASET_DIR, wav_name)
    feature_path = os.path.join(FEATURE_DIR, wav_name.replace(".wav", ".pt"))
    if os.path.exists(feature_path):
        continue
    feats = extract_feature(wav_path)
    torch.save(feats, feature_path)
    print(f"✅ {wav_name} → {feature_path}")

print(f"🔥 Feature extraction завершён. Все файлы сохранены в {FEATURE_DIR}")


In [ ]:
# Скачиваем через huggingface-cli (авторизация не нужна для публичной модели)
!mkdir -p /content/hubert
!wget -O /content/hubert/hubert_base.pt "https://huggingface.co/facebook/hubert-base-ls960/resolve/main/hubert_base.pt"


In [ ]:
!python /content/Retrieval-based-Voice-Conversion-WebUI/infer/modules/train/extract_feature.py \
    /content/Retrieval-based-Voice-Conversion-WebUI/dataset_speaker \
    /content/hubert/hubert_base.pt
